## How to analyse a (Stytra) embedded tail tracking experiment

In this tutorial we show how to use bouter to analyse data that was aquired with [Stytra](http://www.portugueslab.com/stytra/).

First, some imports:

In [1]:
from bouter import EmbeddedExperiment
from bouter import ASSETS_PATH

/Users/luigipetrucco/lab_code/bouter/bouter/tests/assets
/Users/luigipetrucco/lab_code/bouter/bouter/tests/assets


Given a folder containing data from a Stytra experiment, it's very easy to instantiate an `Experiment` object. Here we use example bouter data, otherwise replace the path with the folder to your data:

In [3]:
experiment_data_path = ASSETS_PATH / "embedded_dataset"

# Initialize the experiment:
exp = EmbeddedExperiment(experiment_data_path)

## The `Experiment` class

`Experiment` objects are objects that behaves like dictionariers with extended magical properties.

They have keys, reflecting the Stytra metadata:

In [4]:
print(exp.keys())

dict_keys(['camera_params', 'general', 'gui', 'stimulus', 'behavior'])


And looking inside these keys, we found Stytra hierarchical data log:

In [ ]:
print(exp["general"]["animal"])

In addition, there are some values (*properties*) that are computed from the metadata, invoked in this way:

In [ ]:
# sampling dt of the behavioral tracking
exp.behavior_dt

There's several useful properties. E.g.

* Fish id from folder:

In [ ]:
exp.fish_id

* Unique id for the experiment:

In [ ]:
exp.full_name

* Protocol name:

In [ ]:
exp.protocol_name

* Simplified protocol parameters dict:

In [ ]:
exp.protocol_parameters

* Start and end times of each stimulus in the log:

In [ ]:
(exp.stim_start_times, exp.stim_end_times)

## The behavioural logs

The bulk of the behavioural data is contained in the stytra log files.
With properties you can look at the logs of the experiments, returned as `pd.DataFrame`s; depending on what experiment you run you'll find for example:

 * `Experiment.stimulus_log`: the dynamic log of stytra stimulus

In [ ]:
exp.stimulus_log.head()

 * `Experiment.behavior_log`: the dynamic log of the tracked tail (or eye, position, etc.):

In [ ]:
exp.behavior_log.head()

### Filter the tail log

Sometimes, Stytra will mess the tracking of the last tail segments. Luckily, this can be fixed using a custom interpolation function. It uses n=`continue_curvature` points to average the difference of consecutive segments and fill the nan values using the same incremental difference. This in general is safe as long as the tail tracking was not completely off.

To use it, before using the behavior log, run the following:

In [ ]:
exp.reconstruct_missing_segments(continue_curvature=4)
exp.behavior_log.head()

### Extract and analyse bouts

To analyse bouts, we will need to:

1. Compute vigor
2. Detect bouts
3. (Optional) Extract bout statistics

**1. Compute vigor**: use a rolling standard deviation to infer fish speed. After calling the function, a `exp._behavior_log["vigor"]` column is added to our `exp._behavior_log` dataframe.

`vigor_duration_s` is the window for the rolling std in s; the default 0.05 is the lab standard.

In [ ]:
exp.compute_vigor(vigor_duration_s=0.05)
exp.behavior_log.head()

**2. Detect bouts**: threshold the vigor to find consecutive periods of continuous swimming. # This will return an array with the *index* (not the time) for the detected bouts start and end points.

`vigor_threshold` is the threshold on the vigor in a.u.; 0.1 is a reasonable value that will work in 95% of cases.

In [ ]:
exp.get_bouts(vigor_threshold=0.1)

**3. Extract bouts properties**: Infer some properties of the detected bouts. It Returns a dataframe with bout properties.

`directionality_duration` is a window in seconds to compute directionality idx; lab standard is 0.07 s.

In [ ]:
exp.get_bout_properties(directionality_duration=0.07)

---
**NOTE**

Running exp.get_bout_properties() will work, but just beware that default arguments will be used for all the functions. I.e. at the beginning of your notebook, for a quick analysis you can just type:

```python

exp = EmbeddedExperiment(experiment_data_path)
bouts_df = exp.get_bout_properties()

```
---